Credit: some material here has been adapted from [Sam Roweis](https://www.cs.nyu.edu/home/people/in_memoriam/samroweis.html)' [Linear Algebra Review](http://www.cs.ubc.ca/~murphyk/Teaching/Papers/roweis_linAlgebra.ps).


In [0]:
import numpy as np
import matplotlib.pyplot as plt

# Systems of Linear Equations

A central problem in linear algebra is the solution of a system of linear equations like this:

$$
\begin{align}
3x + 4y + 2z &= 12\\
x + y + z &=5
\end{align}
$$

Typically, we express this system as a single *matrix equation* in the form: $A\mathbf{x} = \mathbf{b}$, where $A$ is an $m$-by-$n$ matrix, $\mathbf{x}$ is an $n$-dimensional column vector, and $\mathbf{b}$ is an $m$-dimensional column vector. The number of unknowns is $n$ and the number of equations or constraints is $m$. Here is another simple example:

$$
\left[ 
\begin{array}{rr}
2 & -1\\
1 & 1
\end{array}
\right] 
\left[
\begin{array}{c}
x_1 \\
x_2
\end{array}
\right]
=
\left[
\begin{array}{c}
1\\
5
\end{array}
\right]
$$

How do we go about "solving" this system of equations? 

* Finding $\mathbf{b}$ given $A$ and $\mathbf{x}$ is easy - just multiply
* For a single $\mathbf{x}$ and $\mathbf{b}$ there are usually a great many matrices $A$ which satisfy the equation
* The only interesting problem left is to find $\mathbf{x}$ given $A$ and $\mathbf{b}$!

This kind of equation is really a problem statement. It says "we applied the function $A$ and generated the output $\mathbf{b}$; what was the input $\mathbf{x}$?"

The matrix $A$ is dictated to us by our problem, and represents our model of how the system we are studying converts inputs to outputs. The vector $\mathbf{b}$ is the output that we observe (or desire) - we know it. The vector $\mathbf{x}$ is the set of inputs that we are trying to find.

There are two ways of thinking about this system of equations. One is *rowwise* as a set of $m$ equations, or constraints that correspond geometrically to $m$ intersecting constraint surfaces:

$$
\left[
\begin{array}{r}
2x_1 - x_2 &= 1\\
x_1 + x_2 &= 5
\end{array}
\right]
$$

The goal is to find the point(s), for example $(x_1, x_2)$ above, which are at the intersection of all the constraint surfaces. In the example above, these surfaces are two lines in a plane. If the lines intersect then there is a solution. If they are parallel, there is not. If they are coincident then there are infinite solutions. In higher dimensions there are more geometric solutions, but in general there can be no solutions, one solution, or infinite solutions.

The other way of thinking about this system is *columnwise* in which we think of the entire system as a single vector relation:

$$
x_1
\left[
\begin{array}{r}
2\\
1
\end{array}
\right]
+
x_2
\left[
\begin{array}{r}
-1\\
1
\end{array}
\right]
=
\left[
\begin{array}{r}
1\\
5
\end{array}
\right]
$$

The goal here is to discover which linear combination(s) $(x_1, x_2)$, if any, of the $n$ column vectors on the left will give the one on the right.

Either way, the matrix equation $A\mathbf{x} = \mathbf{b}$ is an almost ubiquitous problem whose solution comes up again and again in theoretical derivations and in practical data analysis problems. One of the most important applications is the minimization of quadratic energy functions: if $A$ is symmetric positive definite then the quadratic form $\mathbf{x}^TA\mathbf{x} - 2 \mathbf{x}^T\mathbf{b} + c$ is *minimized* at the point where $A\mathbf{x}=\mathbf{b}$. Such quadratic forms arise often in the study of linear models with Gaussian noise since the log likelihood of data under such models is always a matrix quadratic.



## Matrix inversion and Cramer's rule

Let's first consider the case of a single $\mathbf{x}$. As noted above, geometrically we can think of the rows of the system as encoding constraint surfaces in which the solution vector $\mathbf{x}$ must lie and what we are looking for is the point(s) at which these surfaces intersect. Of course, there may be no solution satisfying the equation; then we typically need some way to pick the "best" approximate solution. The constraints may also intersect along an entire line or surface in which case there are an infinity of solutions; once again we would like to think about which one might be best.

Let's consider finding exact solutions first. The most naive thing we could do is just find the inverse of $A$ and solve the equations as follows:

$$
\begin{align}
A^{-1}A\mathbf{x} & = A^{-1} \mathbf{b}\\
I\mathbf{x} &= A^{-1}\mathbf{b}\\
\mathbf{x} &= A^{-1}\mathbf{b}
\end{align}
$$

which is known as *Cramer's rule*.

Let's apply Cramer's rule to the system of equations above:

$$
\left[
\begin{array}{r}
2x_1 - x_2 &= 1\\
x_1 + x_2 &= 5
\end{array}
\right]
$$

In [0]:
A = np.array([[2, -1], [1, 1]])
print(A)
b = np.array([1, 5])
print(b)
x = np.linalg.inv(A).dot(b) #Cramer's rule, slow
print(x)
print(A.dot(np.linalg.inv(A).dot(b))-b) #check

There are several problems with this approach. Most importantly, many interesting functions are not invertible. Beyond that, matrix inversion is a difficult and potentially numerically unstable operation. **Don't invert a matrix unless you really have to!**

## Exact solutions: ``np.linalg.solve``
In cases where the matrix $A$ is square and of full-rank, one can use ``numpy.linalg.solve``. 

Let's return to the same example and solve it more efficiently.

In [0]:
A = np.array([[2, -1], [1, 1]])
b = np.array([1, 5])
x = np.linalg.solve(A, b)
print(x)

Note, though, that this exact method fails when $A$ is not square.

In [0]:
A = np.array([[1, 2, 3],[4, 5, 6]])
print(A)
b = np.array([[5],[6]])
print(b)
x = np.linalg.solve(A, b)
print(x)

## Least squares:  ``np.linalg.lstsq``

In fact, there is a much better way to define what we want as a solution. We will say that we want a solution $\mathbf{x}^*$ which minimizes the error:

$$E = ||A\mathbf{x}^* - \mathbf{b}||^2 = \mathbf{x}^TA^TA\mathbf{x} - 2\mathbf{x}^TA^T\mathbf{b} + \mathbf{b}^T\mathbf{b}$$

This problem is known as *linear least squares*, for obvious reasons. If there is an exact solution (one giving zero error) it will certainly minimize the above cost, but if there is no solution, we can still find the best possible approximation. If we take the matrix derivative of this expression, we can find the best solution:

$$\mathbf{x}^* = (A^TA)^{-1}A^T\mathbf{b}$$

which takes advantage of the fact that even if $A$ is not invertible, $A^TA$ may be.

But what if the problem is degenerate? In other words, what if there is more than one exact solution or more than one inexact solution which all achieve the same minimum error. How can this occur? 

Imagine an equation like this:

$$\left[
\begin{array}{rr}
1 &  -1
\end{array}
\right] \mathbf{x} = 4$$

in which $A = \left[ \begin{array}{rr}
1 &  -1 
\end{array} \right]$. This equation constrains the difference between the two elements of $\mathbf{x}$ to be 4 but the sum of the two elements can be as large or small as we want. 

We can take things one step further to get around this problem. The answer is to ask for the *minimum norm* vector $\mathbf{x}$ that still minimizes the above error. This breaks the degeneracies in both the exact and inexact cases. In terms of the cost function, this corresponds to adding an infinitesimal penalty on $\mathbf{x}^T\mathbf{x}$:

$$
E = \text{lim}_{\epsilon \rightarrow 0} \left[ \mathbf{x}^TA^TA\mathbf{x} - 2\mathbf{x}^TA^T\mathbf{b} + \mathbf{b}^T\mathbf{b} + \epsilon \mathbf{x}^T\mathbf{x} \right]
$$

and the optimal solution becomes

$$
\mathbf{x}^* = \lim_{\epsilon \rightarrow 0}  \left[ \left(A^TA + \epsilon I\right)^{-1}A^T\mathbf{b}\right]
$$

Now, of course actually computing these solutions efficiently and in a numerically stable way is the topic of much study in numerical methods. However, in Python you don't have to worry about any of this, you can just type ``np.linalg.lstsq(A, b)`` and rely on the NumPy maintainers to deliver an optimized solution.

In [0]:
A = np.array([[1, 2, 3],[4, 5, 6]])
print(A)
b = np.array([[5],[6]])
print(b)
# computes the least-squares solution to a linear matrix equation
# equation may be under-, well, or over- determined
x, residuals, rank, s =  np.linalg.lstsq(A, b, rcond=None)
print(x)
print(A.dot(x))

## Reduced-row echelon form

You will notice that we didn't use Gaussian elimination to reduce a matrix to reduced-row echelon form. In fact, numerical linear algebra is very different than the linear algebra we see in textbooks. Reduced-row echelon form doesn't really work when using floating-point values. NumPy, as such, does not have an implementation of Gaussian elimination.

To see a Python implementation, you can head to the SymPy library which does exact computations. It has a good [tutorial on matrix manipulation](https://docs.sympy.org/latest/tutorial/matrices.html).